In [1]:
from sqlalchemy import create_engine, text
import os
from pathlib import Path
from dotenv import load_dotenv

ENV_PATH = Path.cwd() / ".env"
if ENV_PATH.exists():
    load_dotenv(dotenv_path=ENV_PATH, override=True)

DATABASE_URL = os.getenv("POSTGRES_DATABASE")
engine = create_engine(DATABASE_URL)

create_table_sql = """
DROP TABLE IF EXISTS fraud_raw;

CREATE TABLE fraud_raw (
    id                     INT,                  -- première colonne "" du CSV (0,1,2,...)
    trans_date_trans_time  TIMESTAMPTZ,
    cc_num                 BIGINT,
    merchant               TEXT,
    category               TEXT,
    amt                    NUMERIC,
    first                  TEXT,
    last                   TEXT,
    gender                 TEXT,
    street                 TEXT,
    city                   TEXT,
    state                  TEXT,
    zip                    INT,
    lat                    DOUBLE PRECISION,
    long                   DOUBLE PRECISION,
    city_pop               INT,
    job                    TEXT,
    dob                    DATE,
    trans_num              TEXT,
    unix_time              BIGINT,
    merch_lat              DOUBLE PRECISION,
    merch_long             DOUBLE PRECISION,
    is_fraud               INT
);
"""

with engine.begin() as conn:
    conn.execute(text(create_table_sql))

In [2]:
raw_conn = engine.raw_connection()
cur = raw_conn.cursor()

with open("data/fraudTest.csv", "r", encoding="utf-8") as f:
    cur.copy_expert(
        """
        COPY fraud_raw
        FROM STDIN
        WITH CSV HEADER DELIMITER ',';
        """,
        f
    )

raw_conn.commit()
cur.close()
raw_conn.close()

print("✅ Import via COPY terminé")

✅ Import via COPY terminé
